In [1]:
import h2o
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
from h2o.estimators.glrm import H2OGeneralizedLowRankEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
import os
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
import ml_metrics
import sknn
import numpy as np
import string
import itertools

from sklearn.preprocessing import OneHotEncoder

from sklearn.cross_validation import train_test_split
from sknn.mlp import Classifier, Regressor, Layer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression

from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.nonlinearities import softmax
from lasagne.updates import nesterov_momentum,adadelta
from nolearn.lasagne import NeuralNet

from sklearn.ensemble import RandomForestRegressor

from lasagne.nonlinearities import identity
from lasagne.nonlinearities import linear
from nolearn.lasagne import BatchIterator
from nolearn.lasagne  import TrainSplit
from lasagne.updates import adagrad
from nolearn.lasagne import objective
from lasagne.updates import sgd,momentum



from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta
from keras.layers.normalization import BatchNormalization
from keras.callbacks import Callback


h2o.init()
h2o.remove_all() # Clean slate - just in case the cluster was already running



#xgboost
import xgboost as xgb

/home/jma/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)


Using Theano backend.


/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_util.py:67: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/home/jma/anaconda/lib/python2.7/site-packages/scipy/lib/_ut

H2O cluster uptime:,9 days 23 hours 14 minutes 18 seconds 732 milliseconds
H2O cluster version:,3.6.0.8
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,6.94 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


(19765, 895)


(19765, 127)

In [34]:
train=pd.read_csv("/home/jma/Desktop/Data/Kaggle/prudential/train.csv")
test=pd.read_csv("/home/jma/Desktop/Data/Kaggle/prudential/test.csv")

train_rows=train.shape[0]
test_rows=test.shape[0]

y=train.Response

train.drop("Id",axis=1, inplace=True)
test.drop("Id",axis=1, inplace=True)
train.drop("Response",axis=1, inplace=True)

combined=pd.concat([train,test],axis=0)

# In[183]:

train_h2o=h2o.H2OFrame(combined.to_dict('list'))



# In[184]:

cols=pd.Series(train_h2o.col_names)
cols.to_clipboard()
col_loss=["Hinge",
"Quadratic",
"Hinge",
"Quadratic",
"Quadratic",
"Categorical",
"Quadratic",
"Quadratic",
"Quadratic",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Categorical",
"Categorical",
"Categorical",
"Ordinal",
"Categorical",
"Categorical",
"Ordinal",   #changed https://www.kaggle.com/c/prudential-life-insurance-assessment/forums/t/18025/categorical-variables/102307#post102307
"Categorical",
"Hinge",
"Hinge",
"Hinge",
"Quadratic",
"Hinge",
"Hinge",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Quadratic",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Ordinal",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Ordinal",
"Categorical",
"Categorical",
"Categorical",
"Quadratic",
"Quadratic",
"Quadratic",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Ordinal",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Categorical",
"Quadratic",
"Categorical",
"Categorical",
"Categorical",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Hinge",
"Categorical",
"Quadratic"]


# In[185]:
#had to make discrete factors to use ordinal
train_h2o["Product_Info_1"]=train_h2o["Product_Info_1"].asfactor()
train_h2o["Product_Info_2"]=train_h2o["Product_Info_2"].asfactor()
train_h2o["Product_Info_3"]=train_h2o["Product_Info_3"].asfactor()
train_h2o["Product_Info_5"]=train_h2o["Product_Info_5"].asfactor()
train_h2o["Product_Info_6"]=train_h2o["Product_Info_6"].asfactor()
train_h2o["Product_Info_7"]=train_h2o["Product_Info_7"].asfactor()
train_h2o["Employment_Info_2"]=train_h2o["Employment_Info_2"].asfactor()
train_h2o["Employment_Info_3"]=train_h2o["Employment_Info_3"].asfactor()
train_h2o["Employment_Info_5"]=train_h2o["Employment_Info_5"].asfactor()
train_h2o["InsuredInfo_1"]=train_h2o["InsuredInfo_1"].asfactor()
train_h2o["InsuredInfo_2"]=train_h2o["InsuredInfo_2"].asfactor()
train_h2o["InsuredInfo_3"]=train_h2o["InsuredInfo_3"].asfactor()
train_h2o["InsuredInfo_4"]=train_h2o["InsuredInfo_4"].asfactor()
train_h2o["InsuredInfo_5"]=train_h2o["InsuredInfo_5"].asfactor()
train_h2o["InsuredInfo_6"]=train_h2o["InsuredInfo_6"].asfactor()
train_h2o["InsuredInfo_7"]=train_h2o["InsuredInfo_7"].asfactor()
train_h2o["Insurance_History_1"]=train_h2o["Insurance_History_1"].asfactor()
train_h2o["Insurance_History_2"]=train_h2o["Insurance_History_2"].asfactor()
train_h2o["Insurance_History_3"]=train_h2o["Insurance_History_3"].asfactor()
train_h2o["Insurance_History_4"]=train_h2o["Insurance_History_4"].asfactor()
train_h2o["Insurance_History_7"]=train_h2o["Insurance_History_7"].asfactor()
train_h2o["Insurance_History_8"]=train_h2o["Insurance_History_8"].asfactor()
train_h2o["Insurance_History_9"]=train_h2o["Insurance_History_9"].asfactor()
train_h2o["Family_Hist_1"]=train_h2o["Family_Hist_1"].asfactor()
train_h2o["Medical_History_2"]=train_h2o["Medical_History_2"].asfactor()
train_h2o["Medical_History_3"]=train_h2o["Medical_History_3"].asfactor()
train_h2o["Medical_History_4"]=train_h2o["Medical_History_4"].asfactor()
train_h2o["Medical_History_5"]=train_h2o["Medical_History_5"].asfactor()
train_h2o["Medical_History_6"]=train_h2o["Medical_History_6"].asfactor()
train_h2o["Medical_History_7"]=train_h2o["Medical_History_7"].asfactor()
train_h2o["Medical_History_8"]=train_h2o["Medical_History_8"].asfactor()
train_h2o["Medical_History_9"]=train_h2o["Medical_History_9"].asfactor()
train_h2o["Medical_History_10"]=train_h2o["Medical_History_10"].asfactor()
train_h2o["Medical_History_11"]=train_h2o["Medical_History_11"].asfactor()
train_h2o["Medical_History_12"]=train_h2o["Medical_History_12"].asfactor()
train_h2o["Medical_History_13"]=train_h2o["Medical_History_13"].asfactor()
train_h2o["Medical_History_14"]=train_h2o["Medical_History_14"].asfactor()
train_h2o["Medical_History_16"]=train_h2o["Medical_History_16"].asfactor()
train_h2o["Medical_History_17"]=train_h2o["Medical_History_17"].asfactor()
train_h2o["Medical_History_18"]=train_h2o["Medical_History_18"].asfactor()
train_h2o["Medical_History_19"]=train_h2o["Medical_History_19"].asfactor()
train_h2o["Medical_History_20"]=train_h2o["Medical_History_20"].asfactor()
train_h2o["Medical_History_21"]=train_h2o["Medical_History_21"].asfactor()
train_h2o["Medical_History_22"]=train_h2o["Medical_History_22"].asfactor()
train_h2o["Medical_History_23"]=train_h2o["Medical_History_23"].asfactor()
train_h2o["Medical_History_25"]=train_h2o["Medical_History_25"].asfactor()
train_h2o["Medical_History_26"]=train_h2o["Medical_History_26"].asfactor()
train_h2o["Medical_History_27"]=train_h2o["Medical_History_27"].asfactor()
train_h2o["Medical_History_28"]=train_h2o["Medical_History_28"].asfactor()
train_h2o["Medical_History_29"]=train_h2o["Medical_History_29"].asfactor()
train_h2o["Medical_History_30"]=train_h2o["Medical_History_30"].asfactor()
train_h2o["Medical_History_31"]=train_h2o["Medical_History_31"].asfactor()
train_h2o["Medical_History_33"]=train_h2o["Medical_History_33"].asfactor()
train_h2o["Medical_History_34"]=train_h2o["Medical_History_34"].asfactor()
train_h2o["Medical_History_35"]=train_h2o["Medical_History_35"].asfactor()
train_h2o["Medical_History_36"]=train_h2o["Medical_History_36"].asfactor()
train_h2o["Medical_History_37"]=train_h2o["Medical_History_37"].asfactor()
train_h2o["Medical_History_38"]=train_h2o["Medical_History_38"].asfactor()
train_h2o["Medical_History_39"]=train_h2o["Medical_History_39"].asfactor()
train_h2o["Medical_History_40"]=train_h2o["Medical_History_40"].asfactor()
train_h2o["Medical_History_41"]=train_h2o["Medical_History_41"].asfactor()

train_h2o["Medical_History_1"]=train_h2o["Medical_History_1"].asfactor()
train_h2o["Medical_History_15"]=train_h2o["Medical_History_15"].asfactor()
train_h2o["Medical_History_24"]=train_h2o["Medical_History_24"].asfactor()
train_h2o["Medical_History_32"]=train_h2o["Medical_History_32"].asfactor()


# In[186]:

grlmmodel = H2OGeneralizedLowRankEstimator(k = 10,
                                           transform = "STANDARDIZE",
                                           loss_by_col=col_loss,
                                           max_iterations = 50)

grlmmodel.train(x = train_h2o.names , training_frame= train_h2o)
XY=grlmmodel.predict(train_h2o)
XY=XY.as_data_frame()

colNames=[]
for i in xrange(0, len(XY)):
    colNames.append(string.lstrip(XY[i][0],'reconstr_'))
    del XY[i][0]
    
colArrays=np.array(XY[0])

for i in xrange(1,len(colNames)):
     colArrays=np.vstack([colArrays,np.array(XY[i])])

        
grlm_train=pd.DataFrame(np.transpose(colArrays),columns=colNames)

#add the id and Response to the grlm table, these can be anything since they will not overwrite the 100% populated training set
#grlm_train['Id']=0
#grlm_train['Response']=0
grlm_train=grlm_train[combined.columns.values]


# In[193]:

#impute missing from training
pd.set_option('display.max_columns', None)
combined_imp = train.combine_first(grlm_train)



# In[189]:

del train_h2o
del train
del grlm_train
del grlmmodel
del test


# In[234]:
#commented out discrete variables https://www.kaggle.com/c/prudential-life-insurance-assessment/forums/t/18025/categorical-variables/102307#post102307
catCols=[#'Medical_History_1',
         #'Medical_History_15', 
         #'Medical_History_24', 'Medical_History_32',
         'Product_Info_1',
         'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2',
         'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4', 
         'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7',
         'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 
         'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Family_Hist_1', 
         'Medical_History_2', 'Medical_History_3', 'Medical_History_4', 'Medical_History_5', 
         'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 'Medical_History_9', 
         #'Medical_History_10', 
         'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14', 
         'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20',
         'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26', 
         'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_30', 'Medical_History_31', 
         'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37', 
         'Medical_History_38', 'Medical_History_39', 'Medical_History_40', 'Medical_History_41']

pdf=pd.get_dummies(combined_imp[catCols].astype(object))


#catCols.append('Id')
#catCols.append('Response')

listAll=combined_imp.columns.values.tolist()

numCols=[x for x in listAll if x not in catCols]
combinedFinalX=pd.concat([pdf,combined_imp[numCols]],axis=1)
combinedFinalX.head(n=25)



X = StandardScaler().fit_transform(combinedFinalX)
train_X=X[0:train_rows,:]
test_X=X[train_rows:,:]



Parse Progress: [##################################################] 100%

glrm Model Build Progress: [##################################################] 100%


In [16]:
class clsvalidation_kappa(Callback):  #inherits from Callback
    
    def __init__(self, filepath, validation_data=(), patience=5):
        super(Callback, self).__init__()

        self.patience = patience
        self.X_val, self.y_val = validation_data  #tuple of validation X and y
        self.best = 0.0
        self.wait = 0  #counter for patience
        self.filepath=filepath

    def on_epoch_end(self, epoch, logs={}):
        p = self.model.predict(self.X_val, verbose=0)
        current = ml_metrics.quadratic_weighted_kappa(self.y_val.values.ravel(),np.clip(np.round(p.astype(int).ravel()), 1, 8))
        
        if current > self.best:
            self.best = current
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience:
                self.model.stop_training = True
                print('Epoch %d: early stopping' % (epoch))
                print('Retrieving best weights from %s' % (self.filepath))
                self.model.load_weights(self.filepath)
                           
            self.wait += 1 #incremental the number of times without improvement
        print('Epoch %d Kappa: %f | Best Kappa: %f \n' % (epoch,current,self.best))
        

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, y, test_size=0.33, random_state=428)

In [ ]:
val_call = clsvalidation_kappa(validation_data=(X_test, y_test), patience=3, filepath='/home/jma/Desktop/Programming/python_scripts/Kaggle/prudential/keras_models/best.h5') #instantiate object

In [20]:
class NN:
    
    def __init__(self, inputShape, layers, dropout = [], 
                 activation = 'relu', init = 'uniform', loss = 'rmse', 
                 optimizer = 'adadelta', nb_epochs = 50, batch_size = 64, verbose = 1,
                 validation_split=0.0):

        model = Sequential()
        
        for i in range(len(layers)): #loops through layers 
            
            if i == 0:                                                            # The first layer (the input)
                print ("Input shape: " + str(inputShape))
                print ("Adding Layer " + str(i) + ": " + str(layers[i]))
                model.add(Dense(layers[i], input_dim = inputShape, init = init))  #outputs 'layer[i] size' (number of hidden units in the first hidden layer)
                                                                                  #input_dim is number of input units
            
            
            else:                                                                 #is not the first layer (the input)
                print ("Adding Layer " + str(i) + ": " + str(layers[i]))
                model.add(Dense(layers[i], init = init))
            
            print ("Adding " + activation + " layer")
            
            model.add(Activation(activation))
            model.add(BatchNormalization())
            
            if len(dropout) > i:                                                 #drop out still being applied
                print ("Adding " + str(dropout[i]) + " dropout")
                model.add(Dropout(dropout[i]))
        
        model.add(Dense(1, init = init))                                         #End in a single output node for regression style output
        model.compile(loss=loss, optimizer=optimizer)
        
        self.model = model
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.validation_split = validation_split

    def fit(self, X, y): 
        self.model.fit(X, y, nb_epoch=self.nb_epochs, batch_size=self.batch_size, verbose = self.verbose, 
                       callbacks=[val_call],validation_split=self.validation_split)
        
    def predict(self, X, batch_size = 128, verbose = 1):
        return self.model.predict(X, batch_size = batch_size, verbose = verbose)


clf = NN(inputShape = X.shape[1], 
         layers = [512,128], dropout = [0.25, 0.25], 
         loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 10, batch_size =32,
         validation_split=0.0)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

val_call = clsvalidation_kappa(validation_data=(X_test, y_test), patience=300, filepath='/home/jma/Desktop/Programming/python_scripts/Kaggle/prudential/keras_models/best.h5') #instantiate object

print ("Training model...")
clf.fit(X_train,y_train)

print ("Making predictions...")
pred = clf.predict(X_test)
predClipped = np.clip(np.round(pred.astype(int).ravel()), 1, 8) #Make the submissions within the accepted range
ml_metrics.quadratic_weighted_kappa(y_test.values.ravel(),predClipped)
        

Input shape: 895
Adding Layer 0: 512
Adding relu layer
Adding 0.25 dropout
Adding Layer 1: 128
Adding relu layer
Adding 0.25 dropout
Training model...
Epoch 1/10
39785/39785 [==============================] - 20s - loss: 1.5463    
Epoch 0 Kappa: 0.586515 | Best Kappa: 0.586515 

Epoch 2/10
39785/39785 [==============================] - 20s - loss: 1.3837    
Epoch 1 Kappa: 0.569785 | Best Kappa: 0.586515 

Epoch 3/10
39785/39785 [==============================] - 21s - loss: 1.3511    
Epoch 2 Kappa: 0.552831 | Best Kappa: 0.586515 

Epoch 4/10
39785/39785 [==============================] - 23s - loss: 1.3329    
Epoch 3 Kappa: 0.577921 | Best Kappa: 0.586515 

Epoch 5/10
39785/39785 [==============================] - 25s - loss: 1.3138    
Epoch 4 Kappa: 0.602474 | Best Kappa: 0.602474 

Epoch 6/10
39785/39785 [==============================] - 25s - loss: 1.2998    
Epoch 5 Kappa: 0.580025 | Best Kappa: 0.602474 

Epoch 7/10
39785/39785 [==============================] - 26s - loss:

0.5636272595236966

In [19]:
#CV doesnt use early stopping, instead epochs is a parameter to optimise

class NN:
    
    def __init__(self, inputShape, layers, dropout = [], 
                 activation = 'relu', init = 'uniform', loss = 'rmse', 
                 optimizer = 'adadelta', nb_epochs = 50, batch_size = 64, verbose = 0,
                 validation_split=0.0):

        model = Sequential()
        
        for i in range(len(layers)): #loops through layers 
            
            if i == 0:                                                            # The first layer (the input)
                #print ("Input shape: " + str(inputShape))
                #print ("Adding Layer " + str(i) + ": " + str(layers[i]))
                model.add(Dense(layers[i], input_dim = inputShape, init = init))  #outputs 'layer[i] size' (number of hidden units in the first hidden layer)
                                                                                  #input_dim is number of input units
            
            
            else:                                                                 #is not the first layer (the input)
                #print ("Adding Layer " + str(i) + ": " + str(layers[i]))
                model.add(Dense(layers[i], init = init))
            
            #print ("Adding " + activation + " layer")
            
            model.add(Activation(activation))
            model.add(BatchNormalization())
            
            if len(dropout) > i:                                                 #drop out still being applied
                #print ("Adding " + str(dropout[i]) + " dropout")
                model.add(Dropout(dropout[i]))
        
        model.add(Dense(1, init = init))                                         #End in a single output node for regression style output
        model.compile(loss=loss, optimizer=optimizer)
        
        self.model = model
        self.nb_epochs = nb_epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.validation_split = validation_split

    def fit(self, X, y): 
        self.model.fit(X, y, nb_epoch=self.nb_epochs, batch_size=self.batch_size, verbose = self.verbose, 
                       validation_split=self.validation_split)
        
    def predict(self, X, batch_size = 1000, verbose = 0):
        return self.model.predict(X, batch_size = batch_size, verbose = verbose)

    
    

kf = KFold(train_rows, n_folds=3)  #train rows is all training defined above



lst_results=[]


#reduced down to a smaller set to save time
combs = {
  "layers" : [[128,64],[128,128]],
  "dropout" : [[0.5,0.5],[0.25,0.25]],
  "optimizer": ['adadelta'] ,
  "nb_epochs": [5,10],
  "batch_size" :[32,64]
}

params=[[{key: value} for (key, value) in zip(combs, values)]  for values in itertools.product(*combs.values())]

for i in range(len(params)):
    
    layers=params[i][0]['layers']
    nb_epochs=params[i][1]['nb_epochs']
    dropout=params[i][2]['dropout']
    optimizer=params[i][3]['optimizer']
    batch_size=params[i][4]['batch_size']
    
    

    holdScore=[]

    for train_index, test_index in kf:

        #train and test 
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = NN(inputShape = X_train.shape[1], 
             layers = layers, dropout = dropout, 
             loss='mae', optimizer = optimizer, init = 'glorot_normal', nb_epochs = nb_epochs,batch_size = batch_size,
             validation_split=0.0)

        val_call = clsvalidation_kappa(validation_data=(X_test, y_test), patience=3, filepath='/home/jma/Desktop/Programming/python_scripts/Kaggle/prudential/keras_models/best.h5') #instantiate object

        #print ("Training model...")
        clf.fit(X_train,y_train)

        #print ("Making predictions...")
        pred = clf.predict(X_test)
        predClipped = np.clip(np.round(pred.astype(int).ravel()), 1, 8) #Make the submissions within the accepted range
        score=ml_metrics.quadratic_weighted_kappa(y_test.values.ravel(),predClipped)

        holdScore.append(score)

    meanScore= sum(holdScore)/len(holdScore)   
    dict_current={'layers':layers,'dropout': dropout, 'epochs': nb_epochs, 'optimizer' : optimizer, 'batch': batch_size,
                  'meanKappa': meanScore, 'allScores':holdScore }
    
    lst_results.append (dict_current)
    print(dict_current)

    
best=0
for x in lst_results:
    if x['meanKappa'] >best:
        best=x['meanKappa']
        topParam=x
print(topParam)

{'layers': [128, 64], 'optimizer': 'adadelta', 'allScores': [0.5917463591889438, 0.5797231779321281, 0.5816787083153734], 'dropout': [0.5, 0.5], 'batch': 32, 'epochs': 5, 'meanKappa': 0.5843827484788151}
{'layers': [128, 64], 'optimizer': 'adadelta', 'allScores': [0.5743954229746302, 0.5658253125091597, 0.5833221964827997], 'dropout': [0.5, 0.5], 'batch': 64, 'epochs': 5, 'meanKappa': 0.5745143106555298}
{'layers': [128, 64], 'optimizer': 'adadelta', 'allScores': [0.5613979329860594, 0.5571955461016448, 0.5832051279667113], 'dropout': [0.5, 0.5], 'batch': 128, 'epochs': 5, 'meanKappa': 0.567266202351472}
{'layers': [128, 64], 'optimizer': 'adadelta', 'allScores': [0.5946016997558525, 0.565086808629368, 0.5987571109150353], 'dropout': [0.25, 0.25], 'batch': 32, 'epochs': 5, 'meanKappa': 0.586148539766752}
{'layers': [128, 64], 'optimizer': 'adadelta', 'allScores': [0.5928417498832114, 0.5815567377011199, 0.5923843158051889], 'dropout': [0.25, 0.25], 'batch': 64, 'epochs': 5, 'meanKappa'

In [2]:
#keras hyperopt tuning experiment

import numpy as np
np.random.seed(4596) #this, when ran before keras is imported makes keras models reproducable
#https://www.kaggle.com/c/prudential-life-insurance-assessment/forums/t/17829/non-xgboost-success/102018#post102018

import pandas as pd
from sklearn.cross_validation import train_test_split,KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from keras.callbacks import Callback

import ml_metrics

from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.mongoexp import MongoTrials

"""
Load and process (one-hot) both training and test together
Not currently scoring the test set

"""
train=pd.read_csv("/home/jma/Desktop/Data/Kaggle/prudential/train.csv")
test=pd.read_csv("/home/jma/Desktop/Data/Kaggle/prudential/test.csv")

train_rows=train.shape[0]

y=train.Response

train.drop("Id",axis=1, inplace=True)
test.drop("Id",axis=1, inplace=True)
train.drop("Response",axis=1, inplace=True)

combined=pd.concat([train,test],axis=0)


catCols=[
         'Product_Info_1',
         'Product_Info_2', 'Product_Info_3', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7', 'Employment_Info_2',
         'Employment_Info_3', 'Employment_Info_5', 'InsuredInfo_1', 'InsuredInfo_2', 'InsuredInfo_3', 'InsuredInfo_4',
         'InsuredInfo_5', 'InsuredInfo_6', 'InsuredInfo_7',
         'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4',
         'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Family_Hist_1',
         'Medical_History_2', 'Medical_History_3', 'Medical_History_4', 'Medical_History_5',
         'Medical_History_6', 'Medical_History_7', 'Medical_History_8', 'Medical_History_9',
         'Medical_History_11', 'Medical_History_12', 'Medical_History_13', 'Medical_History_14',
         'Medical_History_16', 'Medical_History_17', 'Medical_History_18', 'Medical_History_19', 'Medical_History_20',
         'Medical_History_21', 'Medical_History_22', 'Medical_History_23', 'Medical_History_25', 'Medical_History_26',
         'Medical_History_27', 'Medical_History_28', 'Medical_History_29', 'Medical_History_30', 'Medical_History_31',
         'Medical_History_33', 'Medical_History_34', 'Medical_History_35', 'Medical_History_36', 'Medical_History_37',
         'Medical_History_38', 'Medical_History_39', 'Medical_History_40', 'Medical_History_41']

pdf=pd.get_dummies(combined[catCols].astype(object))

listAll=combined.columns.values.tolist()

numCols=[x for x in listAll if x not in catCols]
X=pd.concat([pdf,combined[numCols]],axis=1)
X.fillna(X.mean(),inplace=True)

X = StandardScaler().fit_transform(X)
train_X=X[0:train_rows,:]
test_X=X[train_rows:,:]



#hold out data from training to use for hyperopt
X_train, X_test, y_train, y_test = train_test_split(train_X, y, test_size=0.20)

#sample out another parition to use for early stopping 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20)



"""
This class is used as a call back with Keras
It will use kappa on the validation data in order to use early stopping,
returning the best model

filepath is where to save the hdf5 table in order to persist the best model

"""
class clsvalidation_kappa(Callback):  #inherits from Callback
    
    def __init__(self, filepath, validation_data=(), patience=5):
        super(Callback, self).__init__()

        self.patience = patience
        self.X_val, self.y_val = validation_data  #tuple of validation X and y
        self.best = 0.0
        self.wait = 0  #counter for patience
        self.filepath=filepath
        self.best_rounds =1
        self.counter=0

    def on_epoch_end(self, epoch, logs={}):
        
        self.counter +=1
        p = self.model.predict(self.X_val, verbose=0) #score the validation data 
        
             
        #current kappa
        current = ml_metrics.quadratic_weighted_kappa(self.y_val.values.ravel(),np.clip(np.round(p.astype(int).ravel()), 1, 8))
       
        print('Epoch %d Kappa: %f | Best Kappa: %f \n' % (epoch,current,self.best))
    
    
        #if improvement over best....
        if current > self.best:
            self.best = current
            self.best_rounds=self.counter
            self.wait = 0
            self.model.save_weights(self.filepath, overwrite=True)
        else:
            if self.wait >= self.patience: #no more patience, retrieve best model
                self.model.stop_training = True
                print('Best number of rounds: %d \nKappa: %f \n' % (self.best_rounds, self.best))
                
                self.model.load_weights(self.filepath)
                           
            self.wait += 1 #incremental the number of times without improvement
        
        

"""
This is the parameter space to explore with hyperopt

Simply offers several discrete choices for numnber of hidden units and drop out rates for
a 2 or 3 layer MLP and also batch size

This can be expanded over other parameters and to sample from a distribution instead of a discrete choice
for # of units etc.

"""
space = {'choice':


hp.choice('num_layers',
    [
                    {'layers':'two',
                     
                                                    
                    },
        
                     {'layers':'three',
                      
                      
                      'units3': hp.choice('units3', [64, 128, 256, 512]),
                      'dropout3': hp.choice('dropout3', [0.25,0.5,0.75])
                                
                    }
        
    
    ]),
    
    'units1': hp.choice('units1', [64, 128, 256, 512]),
    'units2': hp.choice('units2', [64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.25,0.5,0.75]),
    'dropout2': hp.choice('dropout2', [0.25,0.5,0.75]),
    
    'batch_size' : hp.choice('batch_size', [28,64,128]),

    'nb_epochs' :  100,
    'optimizer': 'adadelta',
    'activation': 'relu'
    
    
    }

"""
Objective function that hyperopt will minimize

"""
def objective(params):
    
    import ml_metrics
    import numpy as np
    np.random.seed(4596) #this, when ran before keras is imported makes keras models reproducable
    #https://www.kaggle.com/c/prudential-life-insurance-assessment/forums/t/17829/non-xgboost-success/102018#post102018

    from keras.models import Sequential
    from keras.layers.core import Dense, Dropout, Activation
    from keras.optimizers import Adadelta
    from keras.layers.normalization import BatchNormalization
    from keras.callbacks import Callback
    
    print ('Params testing: ', params)
    print ('\n ')
    model = Sequential()
    model.add(Dense(output_dim=params['units1'], input_dim = X_train.shape[1], init = "glorot_uniform")) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout1']))
    model.add(BatchNormalization())
    
    model.add(Dense(output_dim=params['units2'], init = "glorot_uniform")) 
    model.add(Activation(params['activation']))
    model.add(Dropout(params['dropout2']))
    model.add(BatchNormalization())
    
    if params['choice']['layers']== 'three':
        model.add(Dense(output_dim=params['choice']['units3'], init = "glorot_uniform")) 
        model.add(Activation(params['activation']))
        model.add(Dropout(params['choice']['dropout3']))
        model.add(BatchNormalization())
        patience=25
    else:
        patience=15
    
     
    model.add(Dense(1, init = "glorot_uniform"))    #End in a single output node for regression style output
    model.compile(loss='rmse', optimizer=params['optimizer'])
    
    
    #object of class for call back early stopping 
    val_call = clsvalidation_kappa(validation_data=(X_val, y_val), patience=patience, filepath='/home/jma/Desktop/Programming/python_scripts/Kaggle/prudential/keras_models/best.h5') #instantiate object

    #includes the call back object
    model.fit(X_train, y_train, nb_epoch=params['nb_epochs'], batch_size=params['batch_size'], verbose = 0,callbacks=[val_call])
     
    #predict the test set 
    preds=model.predict(X_test, batch_size = 5000, verbose = 0)
    
    
    predClipped = np.clip(np.round(preds.astype(int).ravel()), 1, 8) #simple rounding of predictionto int
    score=ml_metrics.quadratic_weighted_kappa(y_test.values.ravel(),predClipped)
     
    #adding in params because for some reason they are not being collected properly in trails
    # did fine withoiut the conditional so not sure what is occuring
    return {'loss': score*(-1), 'status': STATUS_OK, 'rounds':val_call.best_rounds, 'params': params }


trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, trials=trials, max_evals=150)

print best
print trials.best_trial



Using Theano backend.
('Params testing: ', {'units1': 128, 'units2': 256, 'optimizer': 'adadelta', 'activation': 'relu', 'batch_size': 64, 'choice': {'layers': 'two'}, 'nb_epochs': 100, 'dropout2': 0.25, 'dropout1': 0.5})

 
Epoch 0 Kappa: 0.538296 | Best Kappa: 0.000000 

Epoch 1 Kappa: 0.582907 | Best Kappa: 0.538296 

Epoch 2 Kappa: 0.553144 | Best Kappa: 0.582907 

Epoch 3 Kappa: 0.562084 | Best Kappa: 0.582907 

Epoch 4 Kappa: 0.582391 | Best Kappa: 0.582907 

Epoch 5 Kappa: 0.547664 | Best Kappa: 0.582907 

Epoch 6 Kappa: 0.552887 | Best Kappa: 0.582907 

Epoch 7 Kappa: 0.586830 | Best Kappa: 0.582907 

Epoch 8 Kappa: 0.576993 | Best Kappa: 0.586830 

Epoch 9 Kappa: 0.559363 | Best Kappa: 0.586830 

Epoch 10 Kappa: 0.564839 | Best Kappa: 0.586830 

Epoch 11 Kappa: 0.606218 | Best Kappa: 0.586830 

Epoch 12 Kappa: 0.553074 | Best Kappa: 0.606218 

Epoch 13 Kappa: 0.585343 | Best Kappa: 0.606218 

Epoch 14 Kappa: 0.578935 | Best Kappa: 0.606218 

Epoch 15 Kappa: 0.593029 | Best Kap

In [4]:
import numpy as np
np.random.seed(4596) #this, when ran before keras is imported makes keras models reproducable
#https://www.kaggle.com/c/prudential-life-insurance-assessment/forums/t/17829/non-xgboost-success/102018#post102018

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adadelta
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(Dense(output_dim=128, input_dim = X_train.shape[1], init = "glorot_uniform")) 
model.add(Activation('relu'))
model.add(Dropout(0.50))
model.add(BatchNormalization())
    
model.add(Dense(output_dim=128, init = "glorot_uniform")) 
model.add(Activation('relu'))
model.add(Dropout(0.75))
model.add(BatchNormalization())

model.add(Dense(output_dim=128, init = "glorot_uniform")) 
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(Dense(1, init = "glorot_uniform"))    #End in a single output node for regression style output
model.compile(loss='rmse', optimizer='adadelta')
    
model.fit(X_train, y_train, nb_epoch=35, batch_size=64, verbose = 0)
     
 
preds=model.predict(X_test, batch_size = 5000, verbose = 0)
predClipped = np.clip(np.round(preds.astype(int).ravel()), 1, 8) #simple rounding of predictionto int
score=ml_metrics.quadratic_weighted_kappa(y_test.values.ravel(),predClipped)
     


In [5]:
print score
import hyperopt.pyll.stochastic
params= hyperopt.pyll.stochastic.sample(space)
print params

0.626762853301
{'units1': 512, 'units2': 64, 'optimizer': 'adadelta', 'activation': 'relu', 'batch_size': 128, 'choice': {'layers': 'two'}, 'nb_epochs': 100, 'dropout2': 0.25, 'dropout1': 0.75}


In [5]:
import sys
sys.path.append('/home/jma/anaconda/lib/python2.7/site-packages/vowpal_wabbit/python')

import pyvw
import sklearn_vw as slvw

In [27]:
pyvw.

[[ 0.  1.]
 [ 0.  2.]
 [ 3.  0.]
 [ 4.  0.]
 [ 1.  2.]
 [ 3.  1.]
 [ 4.  1.]
 [ 3.  2.]
 [ 4.  2.]
 [ 4.  3.]]
  (0, 0)	6.0
  (1, 0)	2.0
  (2, 0)	3.0
  (3, 0)	6.0
  (4, 0)	4.0
  (0, 1)	1.0
  (1, 1)	3.0
  (3, 1)	1.0
  (4, 1)	5.0
